## AIHub Preprocessing Second Review

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

Multiprcessing Library

In [ ]:
%pip install ray

In [1]:
import re
import os
import kss
import ray
import json
from mecab import MeCab
import pandas as pd
from glob import glob
from itertools import chain

In [2]:
pwd

'D:\\AIHUB'

### Function

### 기존 전처리 코드

In [79]:
def first_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거
    
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외) 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외
                
            sentence = re.sub(r"\[.*?\]|\{.*?\}", "",  sentence)
            # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거

            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거
            
            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                preprocessing_sentence_list.append(sentence)

    return preprocessing_sentence_list

### 변경된 전처리 코드

In [210]:
def revised_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}|\<.*?\>", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거
    
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, +, 
            # ①, ②, ③, ④, ⑤, ⑥, ⑦, ⑧, ⑨, ⑩, ⑪, ⑫, ⑬, ⑭, ⑮ 제외) 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외
            
                                    
            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거
            
            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거
            
            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                    
                for sentence2 in kss.split_sentences(sentence, use_heuristic=False,
                                        num_workers=32):
                    for sentence3 in kss.split_sentences(sentence2, use_heuristic=False,
                                                         num_workers=32):
                        preprocessing_sentence_list.append(sentence3)
                        
            # 마지막에 KSS(Korean Sentence Segmentation)로 문장 분리 2번 실행

  
    return preprocessing_sentence_list

# AIHUB 기계독해

#### ko_nia_noanswer_squad_all.json

기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거
<br>(중괄호[], 대괄호{}, 꺽쇠<>)

In [90]:
paragraph1 = "제7호 태풍 &쁘라삐룬&이 점차 우리나라로 북상하고 있습니다. 제주지역은 오늘 밤부터 태풍 영향권에 들 것으로 보이는데요, 서귀포시 법환포구에 나가 있는 중계차 연결합니다.김가람 기자, 현재 상황 알려주시죠.[리포트] 네, 중계차가 나와있는 이곳 서귀포시 법환포구에는 저녁 들어서 바람이 더 강해지고 있습니다. 아직 비는 내리진 않고 있는데요, 해상에 파도가 점차 높게 치면서 태풍 북상을 실감할 수 있습니다. 제주도 모든 해상에는 풍랑주의보가, 제주도 남쪽 먼바다에는 태풍주의보가 발효중입니다. 태풍 특보는 오늘 밤 제주도 서부를 제외한 주변 해상과 육상까지 확대될 가능성이 높습니다. 제주공항을 비롯한 다른 공항의 기상악화와 연결편 문제 등으로 오늘 제주를 오가는 항공기 18편이 결항하고 49편이 지연 운항할 것으로 예상됩니다. 제주와 목포를 잇는 항로를 제외한 대부분 여객선 운항도 중단됐습니다. 기상청은 태풍이 내일 오전 제주도에 가장 가까이 접근할 것으로 예보했습니다. 해수면 수위가 높은 때여서 태풍이 근접해 지나갈 때 침수나 월파 피해 입지 않도록 주의해야 합니다. 오늘 밤 늦게부터 돌풍과 함께 시간당 30mm가 넘는 강한 비가 내리는 곳이 있겠는데요, 모레까지 제주지역 예상 강수량은 50-100mm로 예보됐습니다. 태풍 북상 소식에 제주도 재난안전대책본부는 재난상황실을 가동하고 있고, 학교 10여 곳은 내일 등교 시간을 30분에서 2시간까지 늦추기로 결정했습니다. 지금까지 서귀포시 법환포구에서 KBS 뉴스 김가람입니다."

In [91]:
print(paragraph1)

제7호 태풍 &쁘라삐룬&이 점차 우리나라로 북상하고 있습니다. 제주지역은 오늘 밤부터 태풍 영향권에 들 것으로 보이는데요, 서귀포시 법환포구에 나가 있는 중계차 연결합니다.김가람 기자, 현재 상황 알려주시죠.[리포트] 네, 중계차가 나와있는 이곳 서귀포시 법환포구에는 저녁 들어서 바람이 더 강해지고 있습니다. 아직 비는 내리진 않고 있는데요, 해상에 파도가 점차 높게 치면서 태풍 북상을 실감할 수 있습니다. 제주도 모든 해상에는 풍랑주의보가, 제주도 남쪽 먼바다에는 태풍주의보가 발효중입니다. 태풍 특보는 오늘 밤 제주도 서부를 제외한 주변 해상과 육상까지 확대될 가능성이 높습니다. 제주공항을 비롯한 다른 공항의 기상악화와 연결편 문제 등으로 오늘 제주를 오가는 항공기 18편이 결항하고 49편이 지연 운항할 것으로 예상됩니다. 제주와 목포를 잇는 항로를 제외한 대부분 여객선 운항도 중단됐습니다. 기상청은 태풍이 내일 오전 제주도에 가장 가까이 접근할 것으로 예보했습니다. 해수면 수위가 높은 때여서 태풍이 근접해 지나갈 때 침수나 월파 피해 입지 않도록 주의해야 합니다. 오늘 밤 늦게부터 돌풍과 함께 시간당 30mm가 넘는 강한 비가 내리는 곳이 있겠는데요, 모레까지 제주지역 예상 강수량은 50-100mm로 예보됐습니다. 태풍 북상 소식에 제주도 재난안전대책본부는 재난상황실을 가동하고 있고, 학교 10여 곳은 내일 등교 시간을 30분에서 2시간까지 늦추기로 결정했습니다. 지금까지 서귀포시 법환포구에서 KBS 뉴스 김가람입니다.


### 기존 전처리 코드

In [181]:
sentences = first_preprocessing_text(paragraph1)
for sentence in sentences:
    print(sentence, end="\n\n")

제7호 태풍  쁘라삐룬 이 점차 우리나라로 북상하고 있습니다.

제주지역은 오늘 밤부터 태풍 영향권에 들 것으로 보이는데요, 서귀포시 법환포구에 나가 있는 중계차 연결합니다.

김가람 기자, 현재 상황 알려주시죠. 네, 중계차가 나와있는 이곳 서귀포시 법환포구에는 저녁 들어서 바람이 더 강해지고 있습니다.

아직 비는 내리진 않고 있는데요, 해상에 파도가 점차 높게 치면서 태풍 북상을 실감할 수 있습니다.

제주도 모든 해상에는 풍랑주의보가, 제주도 남쪽 먼바다에는 태풍주의보가 발효중입니다.

태풍 특보는 오늘 밤 제주도 서부를 제외한 주변 해상과 육상까지 확대될 가능성이 높습니다.

제주공항을 비롯한 다른 공항의 기상악화와 연결편 문제 등으로 오늘 제주를 오가는 항공기 18편이 결항하고 49편이 지연 운항할 것으로 예상됩니다.

제주와 목포를 잇는 항로를 제외한 대부분 여객선 운항도 중단됐습니다.

기상청은 태풍이 내일 오전 제주도에 가장 가까이 접근할 것으로 예보했습니다.

해수면 수위가 높은 때여서 태풍이 근접해 지나갈 때 침수나 월파 피해 입지 않도록 주의해야 합니다.

오늘 밤 늦게부터 돌풍과 함께 시간당 30mm가 넘는 강한 비가 내리는 곳이 있겠는데요, 모레까지 제주지역 예상 강수량은 50-100mm로 예보됐습니다.

태풍 북상 소식에 제주도 재난안전대책본부는 재난상황실을 가동하고 있고, 학교 10여 곳은 내일 등교 시간을 30분에서 2시간까지 늦추기로 결정했습니다.

지금까지 서귀포시 법환포구에서 KBS 뉴스 김가람입니다.



### 변경된 전처리 코드

In [133]:
sentences = revised_preprocessing_text(paragraph1)
for sentence in sentences:
    print(sentence, end="\n\n")

제7호 태풍 쁘라삐룬 이 점차 우리나라로 북상하고 있습니다.

제주지역은 오늘 밤부터 태풍 영향권에 들 것으로 보이는데요, 서귀포시 법환포구에 나가 있는 중계차 연결합니다.

네, 중계차가 나와있는 이곳 서귀포시 법환포구에는 저녁 들어서 바람이 더 강해지고 있습니다.

아직 비는 내리진 않고 있는데요, 해상에 파도가 점차 높게 치면서 태풍 북상을 실감할 수 있습니다.

제주도 모든 해상에는 풍랑주의보가, 제주도 남쪽 먼바다에는 태풍주의보가 발효중입니다.

태풍 특보는 오늘 밤 제주도 서부를 제외한 주변 해상과 육상까지 확대될 가능성이 높습니다.

제주공항을 비롯한 다른 공항의 기상악화와 연결편 문제 등으로 오늘 제주를 오가는 항공기 18편이 결항하고 49편이 지연 운항할 것으로 예상됩니다.

제주와 목포를 잇는 항로를 제외한 대부분 여객선 운항도 중단됐습니다.

기상청은 태풍이 내일 오전 제주도에 가장 가까이 접근할 것으로 예보했습니다.

해수면 수위가 높은 때여서 태풍이 근접해 지나갈 때 침수나 월파 피해 입지 않도록 주의해야 합니다.

오늘 밤 늦게부터 돌풍과 함께 시간당 30mm가 넘는 강한 비가 내리는 곳이 있겠는데요, 모레까지 제주지역 예상 강수량은 50-100mm로 예보됐습니다.

태풍 북상 소식에 제주도 재난안전대책본부는 재난상황실을 가동하고 있고, 학교 10여 곳은 내일 등교 시간을 30분에서 2시간까지 늦추기로 결정했습니다.

지금까지 서귀포시 법환포구에서 KBS 뉴스 김가람입니다.



# AIHUB 전문분야 말뭉치

In [186]:
paragraph2 = "제1조(설치 및 기능)① 행정 각 부ㆍ처ㆍ청 간의 협조를 긴밀하게 하며 국무회의에 제출된 의안과 국무회의로부터 지시받은 사항을 심의하기 위하여 차관회의를 둔다. ② 차관회의는 국무에 관하여 국무회의에 건의할 수 있다. [전문개정 2011.11.7]"

#### 법령_l_1000_1.json

3.1) 문장의 시작이 특수문자인 문장
<br>(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, ①, ②,  ③, ④, ⑤, ⑥, ⑦, ⑧, ⑨, ⑩, ⑪, ⑫, ⑬, ⑭, ⑮ 제외) 제외

In [193]:
print(paragraph2)

제1조(설치 및 기능)① 행정 각 부ㆍ처ㆍ청 간의 협조를 긴밀하게 하며 국무회의에 제출된 의안과 국무회의로부터 지시받은 사항을 심의하기 위하여 차관회의를 둔다. ② 차관회의는 국무에 관하여 국무회의에 건의할 수 있다. [전문개정 2011.11.7]


### 기존 전처리 코드

In [211]:
sentences = first_preprocessing_text(paragraph2)
for sentence in sentences:
    print(sentence, end="\n\n")

제1조(설치 및 기능)  행정 각 부 처 청 간의 협조를 긴밀하게 하며 국무회의에 제출된 의안과 국무회의로부터 지시받은 사항을 심의하기 위하여 차관회의를 둔다.



### 변경된 전처리 코드

In [212]:
sentences = revised_preprocessing_text(paragraph2)
for sentence in sentences:
    print(sentence, end="\n\n")

제1조(설치 및 기능) 행정 각 부 처 청 간의 협조를 긴밀하게 하며 국무회의에 제출된 의안과 국무회의로부터 지시받은 사항을 심의하기 위하여 차관회의를 둔다.

차관회의는 국무에 관하여 국무회의에 건의할 수 있다.



# AIHUB 요약문 및 레포트 생성 데이터

### 05.minute

####  REPORT-minute-00002-02042.json

중괄호 앞에 있는 '성명/직함]' 형태 제거

In [214]:
paragraph5 = "교육부장관 서남수] \"그런데 그것을 일반적으로 열어 놨을 경우에는 정말로 또 그런 유혹이……\"\n이에리사 위원] \"저는 그렇게 생각 안 합니다. 그 우려 때문에……  정말 이게 70년대에 만들어진 것이고 두 번에 중간에 95년도에 다시 체육 계열만 차단을 시켰습니다. 이것은 정말 저는 꼭 풀어 주어야 된다라는 강력한 그런 말씀을 드리면서……  장관님 한번 내 아들이 체육특기자인데 공부 잘 합니다 어떻게 하시겠어요? 그런 차원에서 검토를 꼭 해 주십사……  이것이 모든 체육인들의 여망이고 또 체육에 관련된 강의를 하시는 교수님들이 보시면서 느끼는 차원에서의 요구 사항입니다.  꼭 검토되어져서 이번 규제가 풀어져야 된다라는 이런 정서에 맞게 체육인들한테도 원하는 전공 공부를 할 수 있도록 부탁을 드리겠습니다.\"\n교육부장관 서남수] \"예 고민해 보겠습니다.\"\n이에리사 위원] \"장관님 매일 고민만 하시지 말고요. 체육인들한테도 교육부에서 다양한 전공을 좀 해서 자기 인생을 펼칠 수 있도록…… 체육계가 졸업하고 나면 할 게 없어서 코치 감독밖에 못 합니다.  그래서 다른 것도 할 수 있는 그러한 문호를 꼭 열어 주십시오.  시간이 얼마 안 남아서 간단하게 제가……  이번에 대학 특성화 지원하는 사업을 발표하셨지요?\"\n교육부장관 서남수] \"예.\"\n이에리사 위원] \"예체능 계열은 3억으로 한정을 하셨습니다.  너무 섭섭합니다.  어떤 이유에서지요?\"\n"

In [215]:
print(paragraph5)

교육부장관 서남수] "그런데 그것을 일반적으로 열어 놨을 경우에는 정말로 또 그런 유혹이……"
이에리사 위원] "저는 그렇게 생각 안 합니다. 그 우려 때문에……  정말 이게 70년대에 만들어진 것이고 두 번에 중간에 95년도에 다시 체육 계열만 차단을 시켰습니다. 이것은 정말 저는 꼭 풀어 주어야 된다라는 강력한 그런 말씀을 드리면서……  장관님 한번 내 아들이 체육특기자인데 공부 잘 합니다 어떻게 하시겠어요? 그런 차원에서 검토를 꼭 해 주십사……  이것이 모든 체육인들의 여망이고 또 체육에 관련된 강의를 하시는 교수님들이 보시면서 느끼는 차원에서의 요구 사항입니다.  꼭 검토되어져서 이번 규제가 풀어져야 된다라는 이런 정서에 맞게 체육인들한테도 원하는 전공 공부를 할 수 있도록 부탁을 드리겠습니다."
교육부장관 서남수] "예 고민해 보겠습니다."
이에리사 위원] "장관님 매일 고민만 하시지 말고요. 체육인들한테도 교육부에서 다양한 전공을 좀 해서 자기 인생을 펼칠 수 있도록…… 체육계가 졸업하고 나면 할 게 없어서 코치 감독밖에 못 합니다.  그래서 다른 것도 할 수 있는 그러한 문호를 꼭 열어 주십시오.  시간이 얼마 안 남아서 간단하게 제가……  이번에 대학 특성화 지원하는 사업을 발표하셨지요?"
교육부장관 서남수] "예."
이에리사 위원] "예체능 계열은 3억으로 한정을 하셨습니다.  너무 섭섭합니다.  어떤 이유에서지요?"



### 기존 전처리 코드

In [216]:
sentences = first_preprocessing_text(paragraph5)
for sentence in sentences:
    print(sentence, end="\n\n")

이에리사 위원   저는 그렇게 생각 안 합니다.

그 우려 때문에   정말 이게 70년대에 만들어진 것이고 두 번에 중간에 95년도에 다시 체육 계열만 차단을 시켰습니다.

이것은 정말 저는 꼭 풀어 주어야 된다라는 강력한 그런 말씀을 드리면서   장관님 한번 내 아들이 체육특기자인데 공부 잘 합니다 어떻게 하시겠어요

그런 차원에서 검토를 꼭 해 주십사   이것이 모든 체육인들의 여망이고 또 체육에 관련된 강의를 하시는 교수님들이 보시면서 느끼는 차원에서의 요구 사항입니다.

이에리사 위원   장관님 매일 고민만 하시지 말고요.

체육인들한테도 교육부에서 다양한 전공을 좀 해서 자기 인생을 펼칠 수 있도록   체육계가 졸업하고 나면 할 게 없어서 코치 감독밖에 못 합니다.

그래서 다른 것도 할 수 있는 그러한 문호를 꼭 열어 주십시오.

이에리사 위원   예체능 계열은 3억으로 한정을 하셨습니다.



### 변경된 전처리 코드

In [217]:
sentences = revised_preprocessing_text(paragraph5)
for sentence in sentences:
    print(sentence, end="\n\n")

저는 그렇게 생각 안 합니다.

그 우려 때문에 정말 이게 70년대에 만들어진 것이고 두 번에 중간에 95년도에 다시 체육 계열만 차단을 시켰습니다.

이것은 정말 저는 꼭 풀어 주어야 된다라는 강력한 그런 말씀을 드리면서 장관님 한번 내 아들이 체육특기자인데 공부 잘 합니다 어떻게 하시겠어요

그런 차원에서 검토를 꼭 해 주십사 이것이 모든 체육인들의 여망이고 또 체육에 관련된 강의를 하시는 교수님들이 보시면서 느끼는 차원에서의 요구 사항입니다.

장관님 매일 고민만 하시지 말고요.

체육인들한테도 교육부에서 다양한 전공을 좀 해서 자기 인생을 펼칠 수 있도록 체육계가 졸업하고 나면 할 게 없어서 코치 감독밖에 못 합니다.

그래서 다른 것도 할 수 있는 그러한 문호를 꼭 열어 주십시오.

예체능 계열은 3억으로 한정을 하셨습니다.



### 09.literature

#### REPORT-literature-02654-02654.json

마지막에서 다시 KSS(Korean Sentence Segmentation)로 문장 분리 2번 실행

In [98]:
paragraph6 = "그날도 무슨 말 끝에 짚두 없는데 가마닌 그렇게 많이 쳐서 뭣을 하느니, 만성이네는 쉰 장 배당인데 반만 치고 안 친다느니 그런 이야기가 벌어졌었 다. “허, 그래서 쓰나!” 문 서방은 바디를 쉬고 펄쩍뛰 었다. “걔가 학굘 좀 다니더니 너무 아는 체하나보드라. 없는 짚에 바쁜 백성들 한테 가마닐 치울 젠 나라에서두 쓸 데가 있어 그러겠지, 공연히 백성들 들 볶느라구 그럴까.” 그는 이렇게도 말했다. “사람이란 남의 공을 알어야 하느니라. 부모의 공두 알어야 하구, 이웃집 사람의 공두 알어야 하구, 나라 공두 알어야 하구. 바른 대루 말이지만 지 난해 삼 년이나 내리 흉년이 들었을 제 나라에서 그처럼 해주잖었으믄 이 근동만 해두 수백 명 굶어 죽었으리라. 뭐 벼 한 톨 있었다든? 너들은 모르 겠지만 옛날엔 흉년이 들면 그대루 앉아서 굶어 죽었느니라. 있는 놈두 못견 디어났지! 생각하면 지금 세상은 고마우니라. 연전 을축년 장마 때만 해두 몇 만 명이 굶어 죽은 줄 아니? 그런 공을 모르구 가마 좀 짜란다구 이러구 저러구 해? 몹쓸 생각이니라. 나라 공을 알어야지. 고마운 줄 알어야지. 만 성이 그놈 잘못 생각이지.” “주 성네보다두 많이 돌아갔나봐유.” 하고 중식이가 미처 말을 마치기도 전에 문 서방은 말꼬리를 툭 채서, “거 다 못된 생각이지. 그런 일거리가 아니구 나라에서 모찌떡을 남보다 더 줬대두 그 녀석 투정을 할까? 도시 그 애가 못쓸 레라. 제 부모 은혤 모르 는 놈이니 나라 공을 알랴만서두…” 이런 문 서방이다.\n"

In [99]:
print(paragraph6)

그날도 무슨 말 끝에 짚두 없는데 가마닌 그렇게 많이 쳐서 뭣을 하느니, 만성이네는 쉰 장 배당인데 반만 치고 안 친다느니 그런 이야기가 벌어졌었 다. “허, 그래서 쓰나!” 문 서방은 바디를 쉬고 펄쩍뛰 었다. “걔가 학굘 좀 다니더니 너무 아는 체하나보드라. 없는 짚에 바쁜 백성들 한테 가마닐 치울 젠 나라에서두 쓸 데가 있어 그러겠지, 공연히 백성들 들 볶느라구 그럴까.” 그는 이렇게도 말했다. “사람이란 남의 공을 알어야 하느니라. 부모의 공두 알어야 하구, 이웃집 사람의 공두 알어야 하구, 나라 공두 알어야 하구. 바른 대루 말이지만 지 난해 삼 년이나 내리 흉년이 들었을 제 나라에서 그처럼 해주잖었으믄 이 근동만 해두 수백 명 굶어 죽었으리라. 뭐 벼 한 톨 있었다든? 너들은 모르 겠지만 옛날엔 흉년이 들면 그대루 앉아서 굶어 죽었느니라. 있는 놈두 못견 디어났지! 생각하면 지금 세상은 고마우니라. 연전 을축년 장마 때만 해두 몇 만 명이 굶어 죽은 줄 아니? 그런 공을 모르구 가마 좀 짜란다구 이러구 저러구 해? 몹쓸 생각이니라. 나라 공을 알어야지. 고마운 줄 알어야지. 만 성이 그놈 잘못 생각이지.” “주 성네보다두 많이 돌아갔나봐유.” 하고 중식이가 미처 말을 마치기도 전에 문 서방은 말꼬리를 툭 채서, “거 다 못된 생각이지. 그런 일거리가 아니구 나라에서 모찌떡을 남보다 더 줬대두 그 녀석 투정을 할까? 도시 그 애가 못쓸 레라. 제 부모 은혤 모르 는 놈이니 나라 공을 알랴만서두…” 이런 문 서방이다.



### 기존 전처리 코드

In [208]:
sentences = first_preprocessing_text(paragraph6)
for sentence in sentences:
    print(sentence, end="\n\n")

그날도 무슨 말 끝에 짚두 없는데 가마닌 그렇게 많이 쳐서 뭣을 하느니, 만성이네는 쉰 장 배당인데 반만 치고 안 친다느니 그런 이야기가 벌어졌었 다.  허, 그래서 쓰나   문 서방은 바디를 쉬고 펄쩍뛰 었다.  걔가 학굘 좀 다니더니 너무 아는 체하나보드라.

없는 짚에 바쁜 백성들 한테 가마닐 치울 젠 나라에서두 쓸 데가 있어 그러겠지, 공연히 백성들 들 볶느라구 그럴까.  그는 이렇게도 말했다.  사람이란 남의 공을 알어야 하느니라.

부모의 공두 알어야 하구, 이웃집 사람의 공두 알어야 하구, 나라 공두 알어야 하구. 바른 대루 말이지만 지 난해 삼 년이나 내리 흉년이 들었을 제 나라에서 그처럼 해주잖었으믄 이 근동만 해두 수백 명 굶어 죽었으리라.

뭐 벼 한 톨 있었다든  너들은 모르 겠지만 옛날엔 흉년이 들면 그대루 앉아서 굶어 죽었느니라.

연전 을축년 장마 때만 해두 몇 만 명이 굶어 죽은 줄 아니  그런 공을 모르구 가마 좀 짜란다구 이러구 저러구 해  몹쓸 생각이니라.

만 성이 그놈 잘못 생각이지.   주 성네보다두 많이 돌아갔나봐유.  하고 중식이가 미처 말을 마치기도 전에 문 서방은 말꼬리를 툭 채서,  거 다 못된 생각이지.

그런 일거리가 아니구 나라에서 모찌떡을 남보다 더 줬대두 그 녀석 투정을 할까

제 부모 은혤 모르 는 놈이니 나라 공을 알랴만서두   이런 문 서방이다.



### 변경된 전처리 코드

In [201]:
sentences = revised_preprocessing_text(paragraph6)
for sentence in sentences:
    print(sentence, end="\n\n")

그날도 무슨 말 끝에 짚두 없는데 가마닌 그렇게 많이 쳐서 뭣을 하느니, 만성이네는 쉰 장 배당인데 반만 치고 안 친다느니 그런 이야기가 벌어졌었 다.

허, 그래서 쓰나 문 서방은 바디를 쉬고 펄쩍뛰 었다.

걔가 학굘 좀 다니더니 너무 아는 체하나보드라.

없는 짚에 바쁜 백성들 한테 가마닐 치울 젠 나라에서두 쓸 데가 있어 그러겠지, 공연히 백성들 들 볶느라구 그럴까.

그는 이렇게도 말했다.

사람이란 남의 공을 알어야 하느니라.

부모의 공두 알어야 하구, 이웃집 사람의 공두 알어야 하구, 나라 공두 알어야 하구. 바른 대루 말이지만 지 난해 삼 년이나 내리 흉년이 들었을 제 나라에서 그처럼 해주잖었으믄 이 근동만 해두 수백 명 굶어 죽었으리라.

뭐 벼 한 톨 있었다든 너들은 모르 겠지만 옛날엔 흉년이 들면 그대루 앉아서 굶어 죽었느니라.

연전 을축년 장마 때만 해두 몇 만 명이 굶어 죽은 줄 아니 그런 공을 모르구 가마 좀 짜란다구 이러구 저러구 해 몹쓸 생각이니라.

만 성이 그놈 잘못 생각이지.

주 성네보다두 많이 돌아갔나봐유. 하고 중식이가 미처 말을 마치기도 전에 문 서방은 말꼬리를 툭 채서, 거 다 못된 생각이지.

그런 일거리가 아니구 나라에서 모찌떡을 남보다 더 줬대두 그 녀석 투정을 할까

제 부모 은혤 모르 는 놈이니 나라 공을 알랴만서두 이런 문 서방이다.

